In [0]:
!pip install -U -q PyDrive 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1bMDcIwNq19WieDO9D7VOhRCPR7HTCuly'})
downloaded.GetContentFile('Reviews.csv')

In [0]:
import pandas as pd
import re
from nltk.corpus import stopwords
from pickle import dump, load
reviews = pd.read_csv("Reviews.csv", nrows=100000)

In [0]:
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator', 'Score','Time'], 1)
reviews = reviews.reset_index(drop=True) 

In [0]:
contractions = {"ain't": "am not",
                "aren't": "are not / am not",
                "can't": "cannot",
                "can't've": "cannot have",
                "'cause": "because",
                "could've": "could have",
                "couldn't": "could not",
                "couldn't've": "could not have",
                "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not",
                "hadn't": "had not",
                "hadn't've": "had not have",
                "hasn't": "has not",
                "haven't": "have not",
                "he'd": "he had / he would",
                "he'd've": "he would have",
                "he'll": "he shall / he will",
                "he'll've": "he shall have / he will have",
                "he's": "he has / he is",
                "how'd": "how did",
                "how'd'y": "how do you",
                "how'll": "how will",
                "how's": "how has / how is / how does",
                "I'd": "I had / I would",
                "I'd've": "I would have",
                "I'll": "I shall / I will",
                "I'll've": "I shall have / I will have",
                "I'm": "I am",
                "I've": "I have",
                "isn't": "is not",
                "it'd": "it had / it would",
                "it'd've": "it would have",
                "it'll": "it shall / it will",
                "it'll've": "it shall have / it will have",
                "it's": "it has / it is",
                "let's": "let us",
                "ma'am": "madam",
                "mayn't": "may not",
                "might've": "might have",
                "mightn't": "might not",
                "mightn't've": "might not have",
                "must've": "must have",
                "mustn't": "must not",
                "mustn't've": "must not have",
                "needn't": "need not",
                "needn't've": "need not have",
                "o'clock": "of the clock",
                "oughtn't": "ought not",
                "oughtn't've": "ought not have",
                "shan't": "shall not",
                "sha'n't": "shall not",
                "shan't've": "shall not have",
                "she'd": "she had / she would",
                "she'd've": "she would have",
                "she'll": "she shall / she will",
                "she'll've": "she shall have / she will have",
                "she's": "she has / she is",
                "should've": "should have",
                "shouldn't": "should not",
                "shouldn't've": "should not have",
                "so've": "so have",
                "so's": "so as / so is",
                "that'd": "that would / that had",
                "that'd've": "that would have",
                "that's": "that has / that is",
                "there'd": "there had / there would",
                "there'd've": "there would have",
                "there's": "there has / there is",
                "they'd": "they had / they would",
                "they'd've": "they would have",
                "they'll": "they shall / they will",
                "they'll've": "they shall have / they will have",
                "they're": "they are",
                "they've": "they have",
                "to've": "to have",
                "wasn't": "was not",
                "we'd": "we had / we would",
                "we'd've": "we would have",
                "we'll": "we will",
                "we'll've": "we will have",
                "we're": "we are",
                "we've": "we have",
                "weren't": "were not",
                "what'll": "what shall / what will",
                "what'll've": "what shall have / what will have",
                "what're": "what are",
                "what's": "what has / what is",
                "what've": "what have",
                "when's": "when has / when is",
                "when've": "when have",
                "where'd": "where did",
                "where's": "where has / where is",
                "where've": "where have",
                "who'll": "who shall / who will",
                "who'll've": "who shall have / who will have",
                "who's": "who has / who is",
                "who've": "who have",
                "why's": "why has / why is",
                "why've": "why have",
                "will've": "will have",
                "won't": "will not",
                "won't've": "will not have",
                "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have",
                "y'all": "you all",
                "y'all'd": "you all would",
                "y'all'd've": "you all would have",
                "y'all're": "you all are",
                "y'all've": "you all have",
                "you'd": "you had / you would",
                "you'd've": "you would have",
                "you'll": "you shall / you will",
                "you'll've": "you shall have / you will have",
                "you're": "you are",
                "you've": "you have"
                }

In [0]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

In [0]:
import nltk
nltk.download('stopwords')
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")
clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Summaries are complete.
Texts are complete.


In [0]:
stories = list()
for i, text in enumerate(clean_texts):
    stories.append({'story': text, 'highlights': clean_summaries[i]})
dump(stories, open('review_dataset_.pkl', 'wb'))

In [0]:
from google.colab import files
files.download('review_dataset_.pkl') 